# Chapter 10: Advanced Features

## Deletion Vectors Overview

### The Main Idea
Sometimes we can look at a problem and think of different ways to solve it. One of the ways we've done this in Delta Lake is to enable the feature called [deletion vectors](https://docs.delta.io/latest/delta-deletion-vectors.html). One of the things you will should consider in data design thinking about how you want to optimize the performance of a table. In essence, you can optimize for either the table readers or the table writers. It's also important to note that the term deletion vector defines more of the form and function of what the process does rather than how it helps you as a feature. One of the key benefits is that it gives you the ability to do a *merge-on-read* operation. It drastically reduces the performance implications of doing simple, delete operations and instead postpones the performance impact of those delete operations until a more convenient time in the future (but in a rather efficient manner).

### Merge-on-Read Explained
What does merge-on-read mean? Merge-on-read means that instead of going through the operation of rewriting a file at the time of deleting a record or set of records from a particular file we instead make some kind of a note that those records are deleted, and postpone the performance impact of actually performing the delete operation until a time where we can run and optimize command or more complicated update statement. Of course, if someone were to read the table after a merge-on-read operation has been initiated then it will merge during that read operation and that's kind of the point because it allows us to minimize the performance impacts of performing a simple delete operation, or even multiple delete operations, and just perform it at a time later where you are already filtering on the same set of files while reading them and otherwise avoid it doing so in situation's where you don't need it to happen straight away.
Deletion vectors are a way to get this kind of merge-on-read behavior. Put simply, deletion vectors are just a file (or multiple files) adjacent to a data file that allows you to know which records are to be deleted out of the larger data file it sits beside and push that delete (rewrite) operation off to a later point in time that is more efficient and convenient). Alongside in this case is relative, it is part of the Delta Lake file but you will notice in partitioned tables that the deletion vector files sit at the top level rather than within the partition. You can observe this in the coming example.

### Working Data
Our data originally hails from [this repo](https://github.com/nytimes/covid-19-data/issues) but is available in [the repo for this docker image](https://github.com/delta-io/delta-docker).

In [1]:
!rm -rf nyt_covid_19/

In [2]:
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [3]:
spark.sql("drop table if exists nyt")

DataFrame[]

In [4]:
from pyspark.sql.functions import col

(
spark
 .read
 .load("rs/data/COVID-19_NYT/")
 .filter(col("state")=="Florida")
 .filter(col("county").isin(['Hillsborough', 'Pasco', 'Pinellas', 'Sarasota']))
 .repartition("county")
 .write
 .format("delta")
 .partitionBy("county")
 .option("path", "nyt_covid_19/")
 .save()
 )

In [5]:
(
spark
.read
.load("nyt_covid_19/")
.write
.partitionBy("county")
.mode("overwrite")
.format("delta")
.saveAsTable("nyt")
)

In [6]:
spark.sql("""
select
  date,
  county,
  state,
  count(1) as rec_count
from
  nyt
where
  county="Pinellas"
and
  date="2020-03-11"
group by
  date,
  county,
  state
order by
  date
""").show()

+----------+--------+-------+---------+
|      date|  county|  state|rec_count|
+----------+--------+-------+---------+
|2020-03-11|Pinellas|Florida|        1|
+----------+--------+-------+---------+



In [7]:
spark.sql("ALTER TABLE nyt SET TBLPROPERTIES ('delta.enableDeletionVectors' = true);")

DataFrame[]

In [8]:
!tree spark-warehouse/nyt/

spark-warehouse/nyt/
├── county=Hillsborough
│   └── part-00000-751576f9-bacd-4eaa-b747-c3cd64aceacd.c000.snappy.parquet
├── county=Pasco
│   └── part-00003-37880ee0-2ae8-4aa3-b031-408456d9fbc4.c000.snappy.parquet
├── county=Pinellas
│   └── part-00001-280a8daf-82fd-4fb5-95a5-43928a15597c.c000.snappy.parquet
├── county=Sarasota
│   └── part-00002-5b8a4fda-70ad-491d-b9a8-4581d1fc7b40.c000.snappy.parquet
└── _delta_log
    ├── 00000000000000000000.json
    └── 00000000000000000001.json

5 directories, 6 files


In [9]:
spark.sql("""
delete
from
  nyt
where
  county='Pinellas'
and
  date='2020-03-11'
""").show()

+-----------------+
|num_affected_rows|
+-----------------+
|                1|
+-----------------+



In [10]:
!tree spark-warehouse/nyt/

spark-warehouse/nyt/
├── county=Hillsborough
│   └── part-00000-751576f9-bacd-4eaa-b747-c3cd64aceacd.c000.snappy.parquet
├── county=Pasco
│   └── part-00003-37880ee0-2ae8-4aa3-b031-408456d9fbc4.c000.snappy.parquet
├── county=Pinellas
│   └── part-00001-280a8daf-82fd-4fb5-95a5-43928a15597c.c000.snappy.parquet
├── county=Sarasota
│   └── part-00002-5b8a4fda-70ad-491d-b9a8-4581d1fc7b40.c000.snappy.parquet
├── deletion_vector_177d0a71-acb5-4815-a54a-c998b3e0c6cc.bin
└── _delta_log
    ├── 00000000000000000000.json
    ├── 00000000000000000001.json
    └── 00000000000000000002.json

5 directories, 8 files


In [11]:
spark.sql("""
delete
from
  nyt
where
  county='Pasco'
""").show()

+-----------------+
|num_affected_rows|
+-----------------+
|              367|
+-----------------+



In [12]:
spark.sql("""
delete
from
  nyt
where
  date='2020-03-13'
""").show()

+-----------------+
|num_affected_rows|
+-----------------+
|                3|
+-----------------+



In [13]:
!tree spark-warehouse/nyt/

spark-warehouse/nyt/
├── county=Hillsborough
│   └── part-00000-751576f9-bacd-4eaa-b747-c3cd64aceacd.c000.snappy.parquet
├── county=Pasco
│   └── part-00003-37880ee0-2ae8-4aa3-b031-408456d9fbc4.c000.snappy.parquet
├── county=Pinellas
│   └── part-00001-280a8daf-82fd-4fb5-95a5-43928a15597c.c000.snappy.parquet
├── county=Sarasota
│   └── part-00002-5b8a4fda-70ad-491d-b9a8-4581d1fc7b40.c000.snappy.parquet
├── deletion_vector_177d0a71-acb5-4815-a54a-c998b3e0c6cc.bin
├── deletion_vector_5019afe7-8cf6-48cc-85ce-7a0ace4cc554.bin
└── _delta_log
    ├── 00000000000000000000.json
    ├── 00000000000000000001.json
    ├── 00000000000000000002.json
    ├── 00000000000000000003.json
    └── 00000000000000000004.json

5 directories, 11 files


In [14]:
spark.sql("describe history nyt").select("operationMetrics").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|operationMetrics                                                                                                                                                                                                                                                                                                         |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{numRemovedFiles -> 0, numRemovedBytes -> 0, numCop


Notice that each one of our actions created a new table version, even when all we did was enable deletion vectors. If we look at the history results we can get a clearer picture of what happened at each step for the deletions. When we deleted just a single record from the table or multiple records from multiple files we can see in the operation metrics that we added a deletion vector to the table. When we deleted records aligned to entire partition the entire file was marked for removal and so we don't need a deletion vector to be added, it simply becomes stale and will be removed on the next clean up operation. When one of the deletion vectors was being applied to a table which already had a deletion vector being applied to it then it combines those results and marks the original for removal.

In [15]:
spark.sql("optimize nyt").show(truncate=False)

+--------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|path                                        |metrics                                                                                                                                                      |
+--------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|file:/opt/spark/work-dir/spark-warehouse/nyt|{3, 3, {5882, 6307, 6030.0, 3, 18090}, {5893, 6317, 6043.666666666667, 3, 18131}, 3, NULL, 3, 3, 0, false, 0, 0, 1712841241751, 0, 12, 0, NULL, {3, 4}, 6, 6}|
+--------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------

In [16]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [17]:
spark.sql("vacuum nyt retain 0 hours")

Deleted 6 files and directories in a total of 5 directories.


DataFrame[path: string]

In [18]:
!tree spark-warehouse/nyt/

spark-warehouse/nyt/
├── county=Hillsborough
│   └── part-00000-4c40fe6f-f092-4ead-acea-600d67c80dc0.c000.snappy.parquet
├── county=Pasco
├── county=Pinellas
│   └── part-00000-2f40964d-695c-4621-8559-b4f81b459993.c000.snappy.parquet
├── county=Sarasota
│   └── part-00000-2dc1a17b-19da-4f1b-826e-d71b91837d60.c000.snappy.parquet
└── _delta_log
    ├── 00000000000000000000.json
    ├── 00000000000000000001.json
    ├── 00000000000000000002.json
    ├── 00000000000000000003.json
    ├── 00000000000000000004.json
    ├── 00000000000000000005.json
    ├── 00000000000000000006.json
    └── 00000000000000000007.json

5 directories, 11 files


## Constraints and Comments

In [19]:
# Add transaction comments for successive operations
spark.conf.set("spark.databricks.delta.commitInfo.userMetadata", "setting-constraints-and-comments-example")

In [20]:
spark.sql("drop table if exists example_table")

DataFrame[]

In [21]:
spark.sql("""
create table example_table (
  id int comment 'uid column',
  content string not null comment 'payload column for text'
)
using delta
""")

DataFrame[]

In [22]:
spark.sql("alter table example_table alter column id comment 'unique id column'")

DataFrame[]

In [23]:
spark.sql("ALTER TABLE example_table ADD CONSTRAINT id CHECK (id > 0)")

DataFrame[]

In [24]:
spark.sql("describe example_table").show( truncate=False)

+--------+---------+-----------------------+
|col_name|data_type|comment                |
+--------+---------+-----------------------+
|id      |int      |unique id column       |
|content |string   |payload column for text|
+--------+---------+-----------------------+



In [25]:
spark.sql("describe history example_table").show(truncate=False)

+-------+-----------------------+------+--------+--------------+----------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------+----------------------------------------+-----------------------------------+
|version|timestamp              |userId|userName|operation     |operationParameters                                                                                 |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics|userMetadata                            |engineInfo                         |
+-------+-----------------------+------+--------+--------------+----------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------+----------------------------------------+-----------------------------------+
|2      |2024-04-11 13:14

In [26]:
spark.sql("show tblproperties example_table").show(truncate=False)

+----------------------+------+
|key                   |value |
+----------------------+------+
|delta.constraints.id  |id > 0|
|delta.minReaderVersion|1     |
|delta.minWriterVersion|3     |
+----------------------+------+



In [27]:
# Remove annotations to log for successive operations
spark.conf.set("spark.databricks.delta.commitInfo.userMetadata", "null")

In [28]:
spark.sql("insert into example_table (id, content) values (1, 'thingmabob')")

DataFrame[]

In [29]:
# Uncomment the next line to test failing the constraint
# spark.sql("insert into example_table (id, content) values (0, 'widget')")
# Fails with: [DELTA_VIOLATE_CONSTRAINT_WITH_VALUES] CHECK constraint id (id > 0) violated by row with values: - id : 0

In [30]:
# Uncomment the next line to test failing constraint
# spark.sql("insert into example_table (id, content) values (2, null)")
# Fails with: [DELTA_NOT_NULL_CONSTRAINT_VIOLATED] NOT NULL constraint violated for column: content.